In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from shapely.geometry import box, Point, Polygon
import xarray as xr
import rioxarray
from rasterio.features import shapes
import os
import folium
from math import asin, atan2, cos, degrees, radians, sin
from geocube.api.core import make_geocube

root = Path.cwd()

In [73]:
# read in plot data
plot_data = pd.read_csv(root / 'vermont' / 'subplot_info.csv')
plot_data = plot_data.loc[~np.isnan(plot_data['fldLatitude'])]

# convert to geodataframe
geometry = gpd.points_from_xy(plot_data['fldLongitude'], plot_data['fldLatitude'], crs='EPSG:4326')
plot_data['geometry'] = geometry
plots = gpd.GeoDataFrame(plot_data,geometry='geometry',crs='EPSG:4326')

In [74]:
# read tree location csv
tree_info = pd.read_csv(root / 'vermont' / 'tree_info.csv')
tree_info['uniqueTreeID'] = (tree_info['PlotID'].astype(str) + '_' +
    tree_info['SubplotID'].astype(str) + '_' +
    tree_info['TreeID'].astype(str)
)

# read tree measurements csv
stats = pd.read_csv(root / 'vermont' / 'tree_stats.csv')
stats['uniqueTreeID'] = (stats['PlotID'].astype(str) + '_' +
    stats['SubplotID'].astype(str) + '_' +
    stats['TreeID'].astype(str)
)
# drop duplicates; keep latest measurements
stats = stats.sort_values('Year').drop_duplicates(subset='uniqueTreeID', keep='last')


# join trees to measurement info
#trees = stats.merge(tree_info, on='uniqueTreeID', how='inner')
trees = stats.merge(tree_info,on=['PlotID','SubplotID','TreeID'],how='inner')
# retain columns
trees = trees[['PlotID','SubplotID','Year','Status','DBH','uniqueTreeID_x','Species','Azimuth','Distance']]
trees = trees.rename(columns={'DBH':'dbh_inches','uniqueTreeID_x':'uniqueTreeID'})
# filter for live trees
trees = trees.loc[trees['Status']==1]

# join to plot geodataframe
tree_gpd = plots.merge(trees, on=['PlotID', 'SubplotID'], how='left')

In [75]:
print(f'Tree info: {len(set(tree_info['uniqueTreeID']))}\nTree stats: {len(set(stats['uniqueTreeID']))}\nTrees: {len(set(trees['uniqueTreeID']))}\nTree gdf: {len(set(tree_gpd['uniqueTreeID']))}') 

# same number of trees in dataframes; filter out dead trees = 528 left

Tree info: 626
Tree stats: 626
Trees: 528
Tree gdf: 404


In [76]:
# add taxonID column based on species codes
tree_codes = {12:'balsam_fir',97: 'spruce',129:'pine',261:'hemlock',316:'maple',318:'maple',371:'birch',372:'birch',375:'birch',403:'hickory',407:'hickory',409:'hickory',531:'beech',541:'ash',743:'aspen',762:'cherry',802:'oak',832:'oak',833:'oak',837:'oak'}
tree_gpd['taxonID'] = tree_gpd['Species'].map(tree_codes)

In [ ]:
tree_gpd['Distance_meters'] = tree_gpd['Distance'] / 3.281
tree_gpd.head(3)

In [83]:
tree_gpd.columns

Index(['PlotID', 'SubplotID', 'fldLatitude', 'fldLongitude', 'geometry',
       'Year', 'Status', 'dbh_inches', 'uniqueTreeID', 'Species', 'Azimuth',
       'Distance', 'taxonID', 'Distance_meters'],
      dtype='object')

In [85]:
def get_point_at_distance(x,lat_column,lon_column,az_column,dist_column):  
  
    R=6371000  # mean earth radius in meters
    lat1 = radians(x[lat_column])
    lon1 = radians(x[lon_column])
    a = radians(x[az_column])
    d = x[dist_column]

    # mathy math
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )

    return Point(degrees(lon2), degrees(lat2))

# apply function to trees df
tree_gpd['tree_point'] = tree_gpd.apply(lambda x: get_point_at_distance(x=x,lat_column='fldLatitude',lon_column='fldLongitude',az_column='Azimuth',dist_column='Distance_meters'),axis=1)

# set tree point locations as new geometry column
tree_points = tree_gpd.set_geometry('tree_point',crs="EPSG:4326").drop('geometry',axis=1)

In [ ]:
ithica = tree_points.loc[tree_points['fldLongitude']< -75.0]

vermont = tree_points.loc[tree_points['fldLongitude']> -75.0]


In [96]:
vermont.to_file(root / 'output' / 'VERMONT' / 'all_trees_VERMONT.gpkg')

In [98]:
m = folium.Map(location= tuple([43.94091, -72.91433]), zoom_start=11)

# folium.GeoJson(
#     plots,
#     marker=folium.Circle(radius=2, fill_color="red", fill_opacity=0.4, color="red", weight=1)
# ).add_to(m)

folium.GeoJson(
    vermont,
    marker=folium.Circle(radius=2, fill_color="blue", fill_opacity=0.4, color="blue", weight=1),
    tooltip=folium.GeoJsonTooltip(
    fields=["fldLatitude", "fldLongitude", "taxonID"])
).add_to(m)

m